In [1]:
from dataclasses import replace
import pandas as pd
import numpy as np
import pm4py as pm
from sqlalchemy import create_engine
from resources__ import variants
import datetime
import plotly.express as plx

#db_connection
db_provider = "postgresql://"
user = "ubuntu1"
password ="3004222950a"
address  = "20.226.14.116:5432/postgres"
engine = create_engine(db_provider + user + ":" + password  + '@'+ address)


# Read eventlog
event_log = pm.read_xes('Hospital Billing - Event Log.xes.gz')
event_log_df = pm.convert_to_dataframe(event_log)
event_log_df['time:timestamp'] = pd.to_datetime(event_log_df['time:timestamp'],utc=True)
#event_log_df.to_sql('event_log',con = engine,schema='billing',if_exists = 'replace' )
print("Done")

parsing log, completed traces ::   0%|          | 0/100000 [00:00<?, ?it/s]

Done


# Overveiw

In [2]:
#Get Dataframe with traces
df_variants = variants.get_variants_df(event_log)
df_variants['info'] = df_variants['info'].astype(str)
#  df_variants.to_sql(name="variants_total_df",con=engine,if_exists='replace',index =False,schema="billing")


In [ ]:
# Overview Information   --------------- { Cses, Events, Activities, Variants, States, Period }
years = event_log_df['time:timestamp'].dt.year.unique().tolist()
dataframes = []
__dataframes = []
for year in years:
    traces_contained = pm.filter_time_range(event_log,dt1=datetime.datetime(year-1,12,31),dt2=datetime.datetime(year,12,31),mode='traces_contained')
    df_traces_contained = pm.convert_to_dataframe(traces_contained)
    variants_trace = pm.get_variants_as_tuples(traces_contained)
    var_ = []
    data_ = []
    for var,dat in variants_trace.items():
        var_.append(var)
        data_.append(dat)
    df_g = pd.DataFrame({'var':var_,'dat':data_})
    xc = ['variant_' + str(h) for h in np.arange(len(df_g))]
    df_g['v_name'] = xc
    __dataframes.append(df_g)
    events_ = [len(df_traces_contained)]
    total_variants = [len(df_g)]
    activities = [len(df_traces_contained['concept:name'].unique())]
    cases = [len(df_traces_contained['case:concept:name'].unique())]
    states = [len(df_traces_contained['state'].unique())] # ----------------------------------------------------------------This particular event log has states
    df_final = pd.DataFrame({
        'Cases':cases,
        'Events':events_,
        'Activities':activities,
        'Variants': total_variants,
        'States': states, # ----------------------------------------------------------------This particular event log has states
        'year': [year]
    })
    dataframes.append(df_final)

events_ = [len(event_log_df)]
total_variants = [len(df_variants)]
activities = [len(event_log_df['concept:name'].unique())]
cases = [len(event_log_df['case:concept:name'].unique())]
states = [len(event_log_df['state'].unique())] # ----------------------------------------------------------------This particular event log has states
df_final_ = pd.DataFrame({
    'Cases':cases,
    'Events':events_,
    'Activities':activities,
    'Variants': total_variants,
    'States': states, # ----------------------------------------------------------------This particular event log has states
    'year': ['total']
})

years_details_df = pd.concat(dataframes)
total_df_details = pd.concat([years_details_df,df_final_])
total_df_details.to_sql(name='overview_details',con=engine,schema='billing',if_exists="replace",index=False)

In [ ]:
# Month and number of activities
month = event_log_df[['time:timestamp','case:concept:name']]
month['month_year'] = month['time:timestamp'].dt.to_period('M')
month_ = month.groupby('month_year')['case:concept:name'].count().to_frame().reset_index()
month_['month_year'] =  month_['month_year'].astype(str)
month_['month_year'] = pd.to_datetime(month_['month_year'])
month_.to_sql(name='count_month',con=engine, schema='billing',if_exists='replace',index =False)

# Events_per Case
events_per_case_df = event_log_df.groupby(['case:concept:name'])['time:timestamp'].count().to_frame().reset_index()
events_per_case_df['t'] = 'a'
events_per_case_df = events_per_case_df.groupby('time:timestamp')['t'].count().to_frame().reset_index()
events_per_case_df.columns = ['Events per case','Count']
events_per_case_df = events_per_case_df.sort_values('Count',ascending=False)
#events_per_case_df.to_sql(name='events_per_case_df', con=engine,schema='billing',index=False,if_exists='replace')

#Activities per case
activities = event_log_df['concept:name'].unique().tolist()
numbers = []
for activity in activities:
    _ = event_log_df[event_log_df['concept:name'] == activity]
    numbers.append(len(_['case:concept:name'].unique()))
g_df = pd.DataFrame({'activities':activities,'count':numbers})
g_df['%'] = g_df['count'] / len(event_log_df['case:concept:name'].unique()) * 100
#g_df.to_sql(name='activities per case', con=engine,schema='billing',index=False,if_exists='replace')

#Activities per case
activities = event_log_df['concept:name'].unique().tolist()
numbers = []
for activity in activities:
    _ = event_log_df[event_log_df['concept:name'] == activity]
    numbers.append(len(_['case:concept:name'].unique()))
g_df = pd.DataFrame({'activities':activities,'count':numbers})
g_df['percentage'] = g_df['count'] / len(event_log_df['case:concept:name'].unique()) * 100
g_df.to_sql(name='activities_per_case', con=engine,schema='billing',index=False,if_exists='replace')

#Df Canceled
df_cancelled = event_log_df.groupby(['case:concept:name','isCancelled'])['time:timestamp'].count().to_frame().reset_index()
df_cancelled = df_cancelled.drop_duplicates(subset=['case:concept:name'],keep ='last')
df_cancelled =  df_cancelled.groupby('isCancelled')['time:timestamp'].count().to_frame().reset_index()
df_cancelled.to_sql(name = 'df_canceled', con=engine,schema ="billing",index=False,if_exists='replace')

#Df_ Closed
df_closed = event_log_df.groupby(['case:concept:name','isClosed'])['time:timestamp'].count().to_frame().reset_index()
df_closed = df_closed.drop_duplicates(subset=['case:concept:name'],keep ='last')
df_closed =  df_closed.groupby('isClosed')['time:timestamp'].count().to_frame().reset_index()
df_closed.to_sql(name = 'df_closed', con=engine,schema ="billing",index=False,if_exists='replace')



# Process

In [ ]:

def process_level_1():
    #Level 1 
    level_1 = pm.filter_variants(event_log,[df_variants['trace'].to_list()[0]])
    level_1_df = pm.convert_to_dataframe(level_1)
    level_1_heunet = pm.discover_heuristics_net(level_1)
    pm.view_heuristics_net(level_1_heunet)

    # Starting activities & Ending Activities
    start_act_1 = pm.get_start_activities(level_1)
    key_1 = []
    value_1 = []
    for k,v in start_act_1.items():
        key_1.append(k)
        value_1.append(v)
    df_start_act_1 = pd.DataFrame({'Activity':key_1,'number_cases':value_1})
    df_start_act_1.to_sql(name= 'df_start_act_1', schema = 'billing',con= engine,index =False,if_exists='replace')

    end_act_1 = pm.get_end_activities(level_1)
    key_1_e = []
    value_1_e = []
    for k,v in end_act_1.items():
        key_1_e.append(k)
        value_1_e.append(v)
    df_end_act_1 = pd.DataFrame({'Activity':key_1_e,'number_cases':value_1_e})
    df_end_act_1['Percentage'] = df_end_act_1['number_cases'] / len(level_1_df['case:concept:name'].unique()) * 100 
    df_end_act_1.to_sql(name= 'df_end_act_1', schema = 'billing',con= engine,index =False,if_exists='replace')

    #% Ending Activities
    plot_ends_1 = plx.bar(data_frame=df_end_act_1,y = 'Activity',x = 'Percentage')
    #plot_ends_1

    #Business Process Model
    bpmn_1 = pm.discover_bpmn_inductive(level_1)
    pm.view_bpmn(bpmn_1)

    df_variants_level_1 = variants.get_variants_df(level_1)
    df_variants_level_1 =  df_variants_level_1[['trace','len','percentage','Variant_Name']]
    df_variants_level_1['len_trace'] = [len(x) for x in df_variants_level_1['trace'].to_list() ]
    df_variants_level_1.to_sql(name = 'df_variants_level_1', schema = 'billing',con = engine, if_exists='replace',index = False)

    #lenght_variant
    len_variance_1 = plx.bar(df_variants_level_1,x = 'Variant_Name', y = 'len_trace' )
    len_variance_1
    ocurrance_1 = plx.bar(df_variants_level_1,x = 'Variant_Name', y = 'percentage')
    ocurrance_1


    #Petri Net Inductive
    net_inductive_1, im_inductive_1, fm_inductive_1 = pm.discover_petri_net_inductive(level_1)
    pm.view_petri_net(net_inductive_1, im_inductive_1, fm_inductive_1)
    # Petri Net alpha
    net_alpha_1, im_alpha_1, fm_alpha_1 = pm.discover_petri_net_alpha(level_1)
    pm.view_petri_net(net_alpha_1, im_alpha_1, fm_alpha_1)
    #Petrinet alpha plus
    net_alphaplus_1, im_alphaplus_1, fm_alphaplus_1 = pm.discover_petri_net_alpha_plus(level_1)
    pm.view_petri_net(net_alphaplus_1, im_alphaplus_1, fm_alphaplus_1 )

#df_start_act_1 = pd.read_sql(sql = """SELECT * FROM billing.df_start_act_1""",con = engine)
#df_start_act_1.to_csv('dataframes/df_start_act_1.csv',index = False)

#df_end_act_1 = pd.read_sql(sql = """SELECT * FROM billing.df_end_act_1""",con = engine)
#df_end_act_1.to_csv('dataframes/df_end_act_1.csv',index = False)

#df_variants_level_1 = pd.read_sql(sql = """SELECT * FROM billing.df_variants_level_1""",con = engine)
#df_variants_level_1.to_csv('dataframes/df_variants_level_1.csv',index = False)

In [18]:
def process_level_16():
    #Level 1 
    level_16 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:4])
    level_16_df = pm.convert_to_dataframe(level_16)
    level_16_heunet = pm.discover_heuristics_net(level_16)
    pm.view_heuristics_net(level_16_heunet)

    start_act_16 = pm.get_start_activities(level_16)
    key_16 = []
    value_16 = []
    for k,v in start_act_16.items():
        key_16.append(k)
        value_16.append(v)
    df_start_act_16 = pd.DataFrame({'Activity':key_16,'number_cases':value_16})
    df_start_act_16.to_sql(name= 'df_start_act_16', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_16 = pm.get_end_activities(level_16)
    key_16_e = []
    value_16_e = []
    for k,v in end_act_16.items():
        key_16_e.append(k)
        value_16_e.append(v)
    df_end_act_16 = pd.DataFrame({'Activity':key_16_e,'number_cases':value_16_e})
    df_end_act_16['Percentage'] = df_end_act_16['number_cases'] / len(level_16_df['case:concept:name'].unique()) * 100 
    df_end_act_16.to_sql(name= 'df_end_act_16', schema = 'billing',con= engine,index =False,if_exists='replace')

    #% Ending Activities
    plot_ends_16 = plx.bar(data_frame=df_end_act_16,y = 'Activity',x = 'Percentage')
    plot_ends_16

    #Business Process Model
    bpmn_16 = pm.discover_bpmn_inductive(level_16)
    pm.view_bpmn(bpmn_16)


    df_variants_level_16 = variants.get_variants_df(level_16)
    df_variants_level_16 =  df_variants_level_16[['trace','len','percentage','Variant_Name']]
    df_variants_level_16['len_trace'] = [len(x) for x in df_variants_level_16['trace'].to_list() ]
    df_variants_level_16.to_sql(name = 'df_variants_level_16', schema = 'billing',con = engine, if_exists='replace',index = False)

    len_variance_16 = plx.bar(df_variants_level_16,x = 'Variant_Name', y = 'len_trace' )
    #len_variance_16
    ocurrance_16 = plx.bar(df_variants_level_16,x = 'Variant_Name', y = 'percentage')
    ocurrance_16


    #Petri Net Inductive
    net_inductive_16, im_inductive_16, fm_inductive_16 = pm.discover_petri_net_inductive(level_16)
    pm.view_petri_net(net_inductive_16, im_inductive_16, fm_inductive_16)
    # Petri Net alpha
    net_alpha_16, im_alpha_16, fm_alpha_16 = pm.discover_petri_net_alpha(level_16)
    pm.view_petri_net(net_alpha_16, im_alpha_16, fm_alpha_16)
    #Petrinet alpha plus
    net_alphaplus_16, im_alphaplus_16, fm_alphaplus_16 = pm.discover_petri_net_alpha_plus(level_16)
    pm.view_petri_net(net_alphaplus_16, im_alphaplus_16, fm_alphaplus_16 )


#df_start_act_16 = pd.read_sql("""SELECT * FROM billing.df_start_act_16""", con= engine)
#df_end_act_16 = pd.read_sql("""SELECT * FROM billing.df_end_act_16""", con= engine)
#df_variants_level_16 = pd.read_sql("""SELECT * FROM billing.df_variants_level_16""", con= engine)

#df_start_act_16.to_csv('dataframes/df_start_act_16.csv',index= False)
#df_end_act_16.to_csv('dataframes/df_end_act_16.csv',index= False)
#df_variants_level_16.to_csv('dataframes/df_variants_level_16.csv',index= False)


In [ ]:
def process_level_31():
    #Level 31 
    level_31 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:8])
    level_31_df = pm.convert_to_dataframe(level_31)
    level_31_heunet = pm.discover_heuristics_net(level_31)
    pm.view_heuristics_net(level_31_heunet)

    start_act_31 = pm.get_start_activities(level_31)
    key_31 = []
    value_31 = []
    for k,v in start_act_31.items():
        key_31.append(k)
        value_31.append(v)
    df_start_act_31 = pd.DataFrame({'Activity':key_31,'number_cases':value_31})
    df_start_act_31.to_sql(name= 'df_start_act_31', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_31 = pm.get_end_activities(level_31)
    key_31_e = []
    value_31_e = []
    for k,v in end_act_31.items():
        key_31_e.append(k)
        value_31_e.append(v)
    df_end_act_31 = pd.DataFrame({'Activity':key_31_e,'number_cases':value_31_e})
    df_end_act_31['Percentage'] = df_end_act_31['number_cases'] / len(level_31_df['case:concept:name'].unique()) * 100 
    df_end_act_31.to_sql(name= 'df_end_act_31', schema = 'billing',con= engine,index =False,if_exists='replace')


    #% Ending Activities
    plot_ends_31 = plx.bar(data_frame=df_end_act_31,y = 'Activity',x = 'Percentage')
    plot_ends_31

    #Business Process Model
    bpmn_31 = pm.discover_bpmn_inductive(level_31)
    pm.view_bpmn(bpmn_31)


    df_variants_level_31 = variants.get_variants_df(level_31)
    df_variants_level_31 =  df_variants_level_31[['trace','len','percentage','Variant_Name']]
    df_variants_level_31['len_trace'] = [len(x) for x in df_variants_level_31['trace'].to_list() ]
    df_variants_level_31.to_sql(name = 'df_variants_level_31', schema = 'billing',con = engine, if_exists='replace',index = False)



    len_variance_31 = plx.bar(df_variants_level_31,x = 'Variant_Name', y = 'len_trace' )
    len_variance_31
    ocurrance_31 = plx.bar(df_variants_level_31,x = 'Variant_Name', y = 'percentage')
    ocurrance_31


    #Petri Net Inductive
    net_inductive_31, im_inductive_31, fm_inductive_31 = pm.discover_petri_net_inductive(level_31)
    pm.view_petri_net(net_inductive_31, im_inductive_31, fm_inductive_31)
    # Petri Net alpha
    net_alpha_31, im_alpha_31, fm_alpha_31 = pm.discover_petri_net_alpha(level_31)
    pm.view_petri_net(net_alpha_31, im_alpha_31, fm_alpha_31)
    #Petrinet alpha plus
    net_alphaplus_31, im_alphaplus_31, fm_alphaplus_31 = pm.discover_petri_net_alpha_plus(level_31)
    pm.view_petri_net(net_alphaplus_31, im_alphaplus_31, fm_alphaplus_31 )


#df_start_act_31 = pd.read_sql("""SELECT * FROM billing.df_start_act_31""", con= engine)
#df_end_act_31 = pd.read_sql("""SELECT * FROM billing.df_end_act_31""", con= engine)
#df_variants_level_31 = pd.read_sql("""SELECT * FROM billing.df_variants_level_31""", con= engine)

#df_start_act_31.to_csv('dataframes/df_start_act_31.csv',index= False)
#df_end_act_31.to_csv('dataframes/df_end_act_31.csv',index= False)
#df_variants_level_31.to_csv('dataframes/df_variants_level_31.csv',index= False)

In [31]:
def process_level_46():
    #Level 46 
    level_46 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:12])
    level_46_df = pm.convert_to_dataframe(level_46)
    level_46_heunet = pm.discover_heuristics_net(level_46)
    pm.view_heuristics_net(level_46_heunet)

    start_act_46 = pm.get_start_activities(level_46)
    key_46 = []
    value_46 = []
    for k,v in start_act_46.items():
        key_46.append(k)
        value_46.append(v)
    df_start_act_46 = pd.DataFrame({'Activity':key_46,'number_cases':value_46})
    df_start_act_46.to_sql(name= 'df_start_act_46', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_46 = pm.get_end_activities(level_46)
    key_46_e = []
    value_46_e = []
    for k,v in end_act_46.items():
        key_46_e.append(k)
        value_46_e.append(v)
    df_end_act_46 = pd.DataFrame({'Activity':key_46_e,'number_cases':value_46_e})
    df_end_act_46['Percentage'] = df_end_act_46['number_cases'] / len(level_46_df['case:concept:name'].unique()) * 100 
    df_end_act_46.to_sql(name= 'df_end_act_46', schema = 'billing',con= engine,index =False,if_exists='replace')


    #% Ending Activities
    plot_ends_46 = plx.bar(data_frame=df_end_act_46,y = 'Activity',x = 'Percentage')
    plot_ends_46

    #Business Process Model
    bpmn_46 = pm.discover_bpmn_inductive(level_46)
    pm.view_bpmn(bpmn_46)


    df_variants_level_46 = variants.get_variants_df(level_46)
    df_variants_level_46 =  df_variants_level_46[['trace','len','percentage','Variant_Name']]
    df_variants_level_46['len_trace'] = [len(x) for x in df_variants_level_46['trace'].to_list() ]
    df_variants_level_46.to_sql(name = 'df_variants_level_46', schema = 'billing',con = engine, if_exists='replace',index = False)



    len_variance_46 = plx.bar(df_variants_level_46,x = 'Variant_Name', y = 'len_trace' )
    len_variance_46
    ocurrance_46 = plx.bar(df_variants_level_46,x = 'Variant_Name', y = 'percentage')
    ocurrance_46


    #Petri Net Inductive
    net_inductive_46, im_inductive_46, fm_inductive_46 = pm.discover_petri_net_inductive(level_46)
    pm.view_petri_net(net_inductive_46, im_inductive_46, fm_inductive_46)
    # Petri Net alpha
    net_alpha_46, im_alpha_46, fm_alpha_46 = pm.discover_petri_net_alpha(level_46)
    pm.view_petri_net(net_alpha_46, im_alpha_46, fm_alpha_46)
    #Petrinet alpha plus
    net_alphaplus_46, im_alphaplus_46, fm_alphaplus_46 = pm.discover_petri_net_alpha_plus(level_46)
    pm.view_petri_net(net_alphaplus_46, im_alphaplus_46, fm_alphaplus_46 )


#df_start_act_46 = pd.read_sql("""SELECT * FROM billing.df_start_act_46""", con= engine)
#df_end_act_46 = pd.read_sql("""SELECT * FROM billing.df_end_act_46""", con= engine)
#df_variants_level_46 = pd.read_sql("""SELECT * FROM billing.df_variants_level_46""", con= engine)

#df_start_act_46.to_csv('dataframes/df_start_act_46.csv',index= False)
#df_end_act_46.to_csv('dataframes/df_end_act_46.csv',index= False)
#df_variants_level_46.to_csv('dataframes/df_variants_level_46.csv',index= False)



In [ ]:
def process_level_46():
    #Level 61 
    level_61 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:60])
    level_61_df = pm.convert_to_dataframe(level_61)
    level_61_heunet = pm.discover_heuristics_net(level_61)
    pm.view_heuristics_net(level_61_heunet)

    start_act_61 = pm.get_start_activities(level_61)
    key_61 = []
    value_61 = []
    for k,v in start_act_61.items():
        key_61.append(k)
        value_61.append(v)
    df_start_act_61 = pd.DataFrame({'Activity':key_61,'number_cases':value_61})
    df_start_act_61.to_sql(name= 'df_start_act_61', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_61 = pm.get_end_activities(level_61)
    key_61_e = []
    value_61_e = []
    for k,v in end_act_61.items():
        key_61_e.append(k)
        value_61_e.append(v)
    df_end_act_61 = pd.DataFrame({'Activity':key_61_e,'number_cases':value_61_e})
    df_end_act_61['Percentage'] = df_end_act_61['number_cases'] / len(level_61_df['case:concept:name'].unique()) * 100 
    df_end_act_61.to_sql(name= 'df_end_act_61', schema = 'billing',con= engine,index =False,if_exists='replace')


    #% Ending Activities
    plot_ends_61 = plx.bar(data_frame=df_end_act_61,y = 'Activity',x = 'Percentage')
    plot_ends_61

    #Business Process Model
    bpmn_61 = pm.discover_bpmn_inductive(level_61)
    pm.view_bpmn(bpmn_61)


    df_variants_level_61 = variants.get_variants_df(level_61)
    df_variants_level_61 =  df_variants_level_61[['trace','len','percentage','Variant_Name']]
    df_variants_level_61['len_trace'] = [len(x) for x in df_variants_level_61['trace'].to_list() ]
    #df_variants_level_61.to_sql(name = 'df_variants_level_61', schema = 'billing',con = engine, if_exists='replace',index = False)



    len_variance_61 = plx.bar(df_variants_level_61,x = 'Variant_Name', y = 'len_trace' )
    len_variance_61
    ocurrance_61 = plx.bar(df_variants_level_61,x = 'Variant_Name', y = 'percentage')
    ocurrance_61


    #Petri Net Inductive
    net_inductive_61, im_inductive_61, fm_inductive_61 = pm.discover_petri_net_inductive(level_61)
    pm.view_petri_net(net_inductive_61, im_inductive_61, fm_inductive_61)
    # Petri Net alpha
    net_alpha_61, im_alpha_61, fm_alpha_61 = pm.discover_petri_net_alpha(level_61)
    pm.view_petri_net(net_alpha_61, im_alpha_61, fm_alpha_61)
    #Petrinet alpha plus
    net_alphaplus_61, im_alphaplus_61, fm_alphaplus_61 = pm.discover_petri_net_alpha_plus(level_61)
    pm.view_petri_net(net_alphaplus_61, im_alphaplus_61, fm_alphaplus_61 )


#df_start_act_61 = pd.read_sql("""SELECT * FROM billing.df_start_act_61""", con= engine)
#df_end_act_61 = pd.read_sql("""SELECT * FROM billing.df_end_act_61""", con= engine)
#df_variants_level_61 = pd.read_sql("""SELECT * FROM billing.df_variants_level_61""", con= engine)

#df_start_act_61.to_csv('dataframes/df_start_act_61.csv',index= False)
#df_end_act_61.to_csv('dataframes/df_end_act_61.csv',index= False)
#df_variants_level_61.to_csv('dataframes/df_variants_level_61.csv',index= False)

In [20]:
def process_level_76():
    #Level 76 
    level_76 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:120])
    level_76_df = pm.convert_to_dataframe(level_76)
    level_76_heunet = pm.discover_heuristics_net(level_76)
    pm.view_heuristics_net(level_76_heunet)

    start_act_76 = pm.get_start_activities(level_76)
    key_76 = []
    value_76 = []
    for k,v in start_act_76.items():
        key_76.append(k)
        value_76.append(v)
    df_start_act_76 = pd.DataFrame({'Activity':key_76,'number_cases':value_76})
    df_start_act_76.to_sql(name= 'df_start_act_76', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_76 = pm.get_end_activities(level_76)
    key_76_e = []
    value_76_e = []
    for k,v in end_act_76.items():
        key_76_e.append(k)
        value_76_e.append(v)
    df_end_act_76 = pd.DataFrame({'Activity':key_76_e,'number_cases':value_76_e})
    df_end_act_76['Percentage'] = df_end_act_76['number_cases'] / len(level_76_df['case:concept:name'].unique()) * 100 
    df_end_act_76.to_sql(name= 'df_end_act_76', schema = 'billing',con= engine,index =False,if_exists='replace')


    #% Ending Activities
    plot_ends_76 = plx.bar(data_frame=df_end_act_76,y = 'Activity',x = 'Percentage')
    plot_ends_76

    #Business Process Model
    bpmn_76 = pm.discover_bpmn_inductive(level_76)
    pm.view_bpmn(bpmn_76)


    df_variants_level_76 = variants.get_variants_df(level_76)
    df_variants_level_76 =  df_variants_level_76[['trace','len','percentage','Variant_Name']]
    df_variants_level_76['len_trace'] = [len(x) for x in df_variants_level_76['trace'].to_list() ]
    df_variants_level_76.to_sql(name = 'df_variants_level_76', schema = 'billing',con = engine, if_exists='replace',index = False)



    len_variance_76 = plx.bar(df_variants_level_76,x = 'Variant_Name', y = 'len_trace' )
    len_variance_76
    ocurrance_76 = plx.bar(df_variants_level_76,x = 'Variant_Name', y = 'percentage')
    ocurrance_76


    #Petri Net Inductive
    net_inductive_76, im_inductive_76, fm_inductive_76 = pm.discover_petri_net_inductive(level_76)
    pm.view_petri_net(net_inductive_76, im_inductive_76, fm_inductive_76)
    # Petri Net alpha
    net_alpha_76, im_alpha_76, fm_alpha_76 = pm.discover_petri_net_alpha(level_76)
    pm.view_petri_net(net_alpha_76, im_alpha_76, fm_alpha_76)
    #Petrinet alpha plus
    net_alphaplus_76, im_alphaplus_76, fm_alphaplus_76 = pm.discover_petri_net_alpha_plus(level_76)
    pm.view_petri_net(net_alphaplus_76, im_alphaplus_76, fm_alphaplus_76 )


#df_start_act_76 = pd.read_sql("""SELECT * FROM billing.df_start_act_76""", con= engine)
#df_end_act_76 = pd.read_sql("""SELECT * FROM billing.df_end_act_76""", con= engine)
#df_variants_level_76 = pd.read_sql("""SELECT * FROM billing.df_variants_level_76""", con= engine)

#df_start_act_76.to_csv('dataframes/df_start_act_76.csv',index= False)
#df_end_act_76.to_csv('dataframes/df_end_act_76.csv',index= False)
#df_variants_level_76.to_csv('dataframes/df_variants_level_76.csv',index= False)

In [29]:
def process_level_91():

    level_91 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:650])
    level_91_df = pm.convert_to_dataframe(level_91)
    level_91_heunet = pm.discover_heuristics_net(level_91)
    pm.view_heuristics_net(level_91_heunet)

    start_act_91 = pm.get_start_activities(level_91)
    key_91 = []
    value_91 = []
    for k,v in start_act_91.items():
        key_91.append(k)
        value_91.append(v)
    df_start_act_91 = pd.DataFrame({'Activity':key_91,'number_cases':value_91})
    df_start_act_91.to_sql(name= 'df_start_act_91', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_91 = pm.get_end_activities(level_91)
    key_91_e = []
    value_91_e = []
    for k,v in end_act_91.items():
        key_91_e.append(k)
        value_91_e.append(v)
    df_end_act_91 = pd.DataFrame({'Activity':key_91_e,'number_cases':value_91_e})
    df_end_act_91['Percentage'] = df_end_act_91['number_cases'] / len(level_91_df['case:concept:name'].unique()) * 100 
    df_end_act_91.to_sql(name= 'df_end_act_91', schema = 'billing',con= engine,index =False,if_exists='replace')

    #% Ending Activities
    plot_ends_91 = plx.bar(data_frame=df_end_act_91,y = 'Activity',x = 'Percentage')
    plot_ends_91

    #Business Process Model
    bpmn_91 = pm.discover_bpmn_inductive(level_91)
    pm.view_bpmn(bpmn_91)


    df_variants_level_91 = variants.get_variants_df(level_91)
    df_variants_level_91 =  df_variants_level_91[['trace','len','percentage','Variant_Name']]
    df_variants_level_91['len_trace'] = [len(x) for x in df_variants_level_91['trace'].to_list() ]
    df_variants_level_91.to_sql(name = 'df_variants_level_91', schema = 'billing',con = engine, if_exists='replace',index = False)

    len_variance_91 = plx.bar(df_variants_level_91,x = 'Variant_Name', y = 'len_trace' )
    len_variance_91
    ocurrance_91 = plx.bar(df_variants_level_91,x = 'Variant_Name', y = 'percentage')
    ocurrance_91

    #Petri Net Inductive
    net_inductive_91, im_inductive_91, fm_inductive_91 = pm.discover_petri_net_inductive(level_91)
    pm.view_petri_net(net_inductive_91, im_inductive_91, fm_inductive_91)
    # Petri Net alpha
    net_alpha_91, im_alpha_91, fm_alpha_91 = pm.discover_petri_net_alpha(level_91)
    pm.view_petri_net(net_alpha_91, im_alpha_91, fm_alpha_91)
    #Petrinet alpha plus
    net_alphaplus_91, im_alphaplus_91, fm_alphaplus_91 = pm.discover_petri_net_alpha_plus(level_91)
    pm.view_petri_net(net_alphaplus_91, im_alphaplus_91, fm_alphaplus_91 )


#df_start_act_91 = pd.read_sql("""SELECT * FROM billing.df_start_act_91""", con= engine)
#df_end_act_91 = pd.read_sql("""SELECT * FROM billing.df_end_act_91""", con= engine)
#df_variants_level_91 = pd.read_sql("""SELECT * FROM billing.df_variants_level_91""", con= engine)

#df_start_act_91.to_csv('dataframes/df_start_act_91.csv',index= False)
#df_end_act_91.to_csv('dataframes/df_end_act_91.csv',index= False)
#df_variants_level_91.to_csv('dataframes/df_variants_level_91.csv',index= False)

In [ ]:
def process_level_91():
    level_100 = pm.filter_variants(event_log,df_variants['trace'].to_list())
    level_100_df = pm.convert_to_dataframe(level_100)
    level_100_heunet = pm.discover_heuristics_net(level_100)
    pm.view_heuristics_net(level_100_heunet)

    start_act_100 = pm.get_start_activities(level_100)
    key_100 = []
    value_100 = []
    for k,v in start_act_100.items():
        key_100.append(k)
        value_100.append(v)
    df_start_act_100 = pd.DataFrame({'Activity':key_100,'number_cases':value_100})
    #df_start_act_100.to_sql(name= 'df_start_act_100', schema = 'billing',con= engine,index =False,if_exists='replace')


    end_act_100 = pm.get_end_activities(level_100)
    key_100_e = []
    value_100_e = []
    for k,v in end_act_100.items():
        key_100_e.append(k)
        value_100_e.append(v)
    df_end_act_100 = pd.DataFrame({'Activity':key_100_e,'number_cases':value_100_e})
    df_end_act_100['Percentage'] = df_end_act_100['number_cases'] / len(level_100_df['case:concept:name'].unique()) * 100 
    #df_end_act_100.to_sql(name= 'df_end_act_100', schema = 'billing',con= engine,index =False,if_exists='replace')

    #% Ending Activities
    plot_ends_100 = plx.bar(data_frame=df_end_act_100,y = 'Activity',x = 'Percentage')
    plot_ends_100

    #Business Process Model
    bpmn_100 = pm.discover_bpmn_inductive(level_100)
    pm.view_bpmn(bpmn_100)


    df_variants_level_100 = variants.get_variants_df(level_100)
    df_variants_level_100 =  df_variants_level_100[['trace','len','percentage','Variant_Name']]
    df_variants_level_100['len_trace'] = [len(x) for x in df_variants_level_100['trace'].to_list() ]
    df_variants_level_100.to_sql(name = 'df_variants_level_100', schema = 'billing',con = engine, if_exists='replace',index = False)

    len_variance_100 = plx.bar(df_variants_level_100,x = 'Variant_Name', y = 'len_trace' )
    len_variance_100
    ocurrance_100 = plx.bar(df_variants_level_100,x = 'Variant_Name', y = 'percentage')
    ocurrance_100

    #Petri Net Inductive
    net_inductive_100, im_inductive_100, fm_inductive_100 = pm.discover_petri_net_inductive(level_100)
    pm.view_petri_net(net_inductive_100, im_inductive_100, fm_inductive_100)
    # Petri Net alpha
    net_alpha_100, im_alpha_100, fm_alpha_100 = pm.discover_petri_net_alpha(level_100)
    pm.view_petri_net(net_alpha_100, im_alpha_100, fm_alpha_100)
    #Petrinet alpha plus
    net_alphaplus_100, im_alphaplus_100, fm_alphaplus_100 = pm.discover_petri_net_alpha_plus(level_100)
    pm.view_petri_net(net_alphaplus_100, im_alphaplus_100, fm_alphaplus_100 )



#df_start_act_100 = pd.read_sql("""SELECT * FROM billing.df_start_act_100""", con= engine)
#df_end_act_100 = pd.read_sql("""SELECT * FROM billing.df_end_act_100""", con= engine)
#df_variants_level_100 = pd.read_sql("""SELECT * FROM billing.df_variants_level_100""", con= engine)

#df_start_act_100.to_csv('dataframes/df_start_act_100.csv',index= False)
#df_end_act_100.to_csv('dataframes/df_end_act_100.csv',index= False)
#df_variants_level_100.to_csv('dataframes/df_variants_level_100.csv',index= False)

# Timing


In [200]:

level_1 = pm.filter_variants(event_log,[df_variants['trace'].to_list()[0]])
level_16 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:4])
level_31 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:8])
level_46 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:12])
level_61 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:60])
level_76 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:120])
level_91 = pm.filter_variants(event_log,df_variants['trace'].to_list()[0:650])
level_100 = pm.filter_variants(event_log,df_variants['trace'].to_list())

In [ ]:
from cProfile import label
from tkinter import Label


groups = [level_1,level_16,level_31,level_46,level_61,level_76,level_91,level_100 ]
labels = ['_1','_16','_31','_46','_61','_76','_91','_100']


for group, label_ in zip(groups,labels):
    net, start, end = pm.discover_performance_dfg(group)
    pm.save_vis_performance_dfg(dfg =net,start_activities=start,end_activities=end,file_path='images/timing/DFG'+label_+'.png',bgcolor="white")
    print('saved' + label_)

In [251]:
min_time = event_log_df['time:timestamp'].min()
max_time = event_log_df['time:timestamp'].max()

days = (max_time-min_time).days
_seconds = (max_time-min_time).seconds
hours =  int(_seconds/3600)
_minutes = (_seconds/3600) - hours
minutes = int(_minutes * 60)

time_head_ = pd.DataFrame({'min_time':[min_time],'max_time':[max_time],'days':[days],'hours':[hours],'minutes':[minutes]})
#time_head_.to_sql(name = 'time_head',schema='billing',con=engine,if_exists='replace',index=False)
#time_head_.to_csv('dataframes/time_head.csv')
#time_head_ = pd.read_sql("""SELECT * FROM billing.time_head""",con= engine)

In [8]:

cases_df_list =event_log_df['case:concept:name'].drop_duplicates().to_frame().reset_index().drop('index',axis =1)
cases_df_list.to_sql(name = "cases_df_list",schema="billing",if_exists="replace",index =False, con  = engine)
cases_df_list.to_csv('dataframes/cases_df_list.csv', index = False)

In [ ]:
cases_dfs = []

for case in cases_df_list['case:concept:name']:
    frame = event_log_df[event_log_df['case:concept:name'] == case][['time:timestamp','concept:name','case:concept:name']]
    interval = frame['time:timestamp'].diff().shift(-1).fillna(datetime.timedelta(0)).apply(lambda y : y.total_seconds()/3600)
    frame['interval'] = interval
    cases_dfs.append(frame)
    print(frame['case:concept:name'].drop_duplicates().to_list()[0])


In [22]:
event_log_df

,isCancelled,diagnosis,time:timestamp,caseType,speciality,org:resource,concept:name,blocked,isClosed,flagD,...,lifecycle:transition,case:concept:name,closeCode,actRed,actOrange,flagC,msgCount,version,msgType,msgCode
0,False,A,2012-12-16 18:33:10+00:00,A,A,ResA,NEW,False,True,True,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2013-12-15 18:00:37+00:00,NaN,NaN,NaN,FIN,NaN,NaN,NaN,...,complete,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2013-12-16 02:53:38+00:00,NaN,NaN,NaN,RELEASE,NaN,NaN,NaN,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2013-12-17 11:56:29+00:00,NaN,NaN,NaN,CODE OK,NaN,NaN,NaN,...,complete,A,NaN,False,False,False,0.0,A,NaN,NaN
4,NaN,NaN,2013-12-19 02:44:31+00:00,NaN,NaN,ResB,BILLED,NaN,NaN,NaN,...,complete,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451354,False,OM,2015-12-13 18:31:23+00:00,A,E,ResA,NEW,False,False,False,...,complete,AXQE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451355,False,NaN,2016-01-14 20:17:47+00:00,B,L,ResDJ,NEW,False,False,False,...,complete,BXQE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451356,True,NaN,2016-01-14 21:00:13+00:00,NaN,NaN,ResFR,DELETE,NaN,NaN,NaN,...,complete,BXQE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451357,False,LL,2016-01-11 18:40:47+00:00,A,D,ResA,NEW,False,False,False,...,complete,CXQE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
cases_df = pd.concat(cases_dfs)
#cases_df.to_sql(con = engine, schema="billing",index = False,name="cases_df", if_exists="replace")


359

In [6]:
event_log_df.head(150).to_csv("dataframes/sample_df.csv")

In [4]:
sql = """ SELECT 
	"concept:name",
	MIN("interval") AS min,
	MAX("interval") AS max,
	AVG("interval") AS mean,
	STDDEV("interval") AS std,
	PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY "interval") AS q1,
	PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY "interval") AS q3
FROM 
	billing.full_case_variant_time 
WHERE "Variant" IN ('Variant_947') 
GROUP BY "concept:name" """


xc = pd.read_sql(con = engine, sql = sql )

In [7]:
import pyautogui
import time
import numpy as np

time.sleep(10)

for i in np.arange(0,15000):
    pyautogui.typewrite(str(i))
    pyautogui.press("enter")
    time.sleep(5)

KeyboardInterrupt: 